### Alternative Model 2
Will try adding a layer, use geometric mean ceil times two for the number of hidden nodes, and 80 epochs.
I used 8 nodes in layer 3.

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from datetime import date

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(
    Path('Resources/applicants_data.csv',)
)

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [4]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])

# Review the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [5]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=='object'].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [7]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [8]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [9]:
non_categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes!='object'].index)
non_categorical_variables

['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']

In [10]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df =pd.concat([applicant_data_df[non_categorical_variables], encoded_df], axis=1)

# Review the Dataframe
encoded_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
X.head()


,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Step 6: Split the features and target sets into training and testing datasets.


In [13]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y)


### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


#### Alternative Model 2
Will try adding a layer, and geometric mean ceil for the number of hidden nodes

In [15]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [16]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = np.ceil(np.sqrt(number_input_features * number_output_neurons_A2))*2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

22.0

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = np.ceil(np.sqrt(hidden_nodes_layer1_A2 * number_output_neurons_A2))*2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A2

10.0

In [19]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A2 = np.ceil(np.sqrt(hidden_nodes_layer2_A2 * number_output_neurons_A2))*2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer3_A2

8.0

In [20]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [21]:
# First hidden layer
nn_A2.add(
    Dense(
        units=hidden_nodes_layer1_A2,
        activation='relu',
        input_dim=number_input_features
    )
)

# Second hidden layer
nn_A2.add(
    Dense(
        units=hidden_nodes_layer2_A2,
        activation='relu'
    )
)


# Third hidden layer
nn_A2.add(
    Dense(
        units=hidden_nodes_layer3_A2,
        activation='relu'
    )
)
# Output layer
nn_A2.add(
    Dense(
        units=1,
        activation='sigmoid'
    )
)

# Check the structure of the model
nn_A2.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 22)                2574      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                230       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 2,901
Trainable params: 2,901
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_A2.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
        
)


In [23]:
# Fit the model
epochs=80
nn_A2.fit(
    X_train_scaled,
    y_train,
    epochs=epochs
)


Epoch 1/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5876 - accuracy: 0.7100
Epoch 2/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7306
Epoch 3/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7322
Epoch 4/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7327
Epoch 5/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7335
Epoch 6/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7340
Epoch 7/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5442 - accuracy: 0.7351
Epoch 8/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7347
Epoch 9/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5428 - accuracy: 0.7345
Epoch 10/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5422 - accuracy: 0.7353

In [24]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(
                                        X_test_scaled,
                                        y_test,
                                        verbose=2
)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5578 - accuracy: 0.7308
Loss: 0.5578314065933228, Accuracy: 0.7308454513549805


In [28]:
# Display the model loss and accuracy results
results_df=pd.DataFrame(
                    data={
                        'Date': [date.today()],
                        'Model'   : [2],
                        'Observations':['Adding a hidden layer, increasing epochs'],
                        'Loss'    : [model_loss], 
                        'Accuracy': [model_accuracy],
                        'Hidden layers': 3,
                        'Input Features':[number_input_features],
                        'Output Neurons': [number_output_neurons_A2],
                        'Neurons in Hidden Layers1' : [hidden_nodes_layer1_A2],
                        'Neurons in Hidden Layers2' : [hidden_nodes_layer2_A2],
                        'Epochs'         : [epochs],
                        'Activation Hidden layer 1': ['relu'],
                        'Activation Hidden layer 2': ['relu'],
                        'Activation Output layer 1': ['sigmoid'],
                        'loss':['binary_crossentropy'],
                        'optimizer':['adam'],
                        'metrics':['accuracy'],
                        'Neurons in Hidden Layers3' : [hidden_nodes_layer3_A2],
                        'Activation Hidden layer 3': ['relu']
                    })

print(results_df.T)

                                                                  0
Date                                                     2021-09-26
Model                                                             2
Observations               Adding a hidden layer, increasing epochs
Loss                                                       0.557831
Accuracy                                                   0.730845
Hidden layers                                                     3
Input Features                                                  116
Output Neurons                                                    1
Neurons in Hidden Layers1                                      22.0
Neurons in Hidden Layers2                                      10.0
Epochs                                                           80
Activation Hidden layer 1                                      relu
Activation Hidden layer 2                                      relu
Activation Output layer 1                       

In [29]:
results_df.to_csv('Resources/results.csv', mode='a') 

### Step 3: Save each of your alternative models as an HDF5 file.


In [27]:
# Set the model's file path
file_path = Path('Resources/AlphabetSoup_Model_2.h5')

# Export your model to a HDF5 file
nn_A2.save(file_path)